# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Data 
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Visualization 
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
df = pd.read_csv("output_data/cities.csv")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Yellowknife,62.4560,-114.3525,-19.44,76,20,6.91,1037,CA,1732766201
1,1,Labrador City,52.9463,-66.9114,31.17,88,100,2.30,1006,CA,1732766743
2,2,Wailua Homesteads,22.0669,-159.3780,72.12,78,1,12.21,1016,US,1732766489
3,3,Anadyr,64.7500,177.4833,30.70,86,40,26.84,1023,RU,1732766681
4,4,Thompson,55.7435,-97.8558,8.76,92,75,2.30,1020,CA,1732766592


In [5]:
# Date Cleaning w/dates
df["Date"] = pd.to_datetime(df.Date * 1e9)
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Yellowknife,62.4560,-114.3525,-19.44,76,20,6.91,1037,CA,2024-11-28 03:56:41
1,1,Labrador City,52.9463,-66.9114,31.17,88,100,2.30,1006,CA,2024-11-28 04:05:43
2,2,Wailua Homesteads,22.0669,-159.3780,72.12,78,1,12.21,1016,US,2024-11-28 04:01:29
3,3,Anadyr,64.7500,177.4833,30.70,86,40,26.84,1023,RU,2024-11-28 04:04:41
4,4,Thompson,55.7435,-97.8558,8.76,92,75,2.30,1020,CA,2024-11-28 04:03:12


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     598 non-null    int64         
 1   City        598 non-null    object        
 2   Lat         598 non-null    float64       
 3   Lng         598 non-null    float64       
 4   Max Temp    598 non-null    float64       
 5   Humidity    598 non-null    int64         
 6   Cloudiness  598 non-null    int64         
 7   Wind Speed  598 non-null    float64       
 8   Pressure    598 non-null    int64         
 9   Country     594 non-null    object        
 10  Date        598 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 51.5+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)
# Display the map
map_plot 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 65
max_temp = 75
max_wind = 10

# Drop any rows with null values
df2 = df.dropna()

mask = (df2["Max Temp"] >= min_temp) & (df2["Max Temp"] < max_temp) & (df2["Wind Speed"] < max_wind)
df2 = df2.loc[mask].reset_index(drop=True)

# Display sample data
df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,11,Margaret River,-33.9500,115.0667,72.93,46,0,8.34,1018,AU,2024-11-28 04:05:54
1,14,Pisco,-13.7000,-76.2167,66.25,88,0,8.05,1014,PE,2024-11-28 04:05:57
2,15,Zemio,5.0314,25.1361,68.14,49,52,3.27,1015,CF,2024-11-28 04:06:00
3,21,Puerto Ayora,-0.7393,-90.3518,71.55,92,95,8.99,1013,EC,2024-11-28 04:06:07
4,35,Dunedin,-45.8742,170.5036,68.81,68,100,5.99,1005,NZ,2024-11-28 03:59:41
...,...,...,...,...,...,...,...,...,...,...,...
69,562,Botro,7.8525,-5.3106,68.95,44,0,5.08,1012,CI,2024-11-28 04:16:28
70,567,Petatlán,17.5167,-101.2667,73.81,92,0,3.40,1012,MX,2024-11-28 04:16:34
71,581,Guiping,23.3925,110.0814,70.79,25,75,6.69,1025,CN,2024-11-28 04:16:50
72,582,Inírida,3.8653,-67.9239,73.99,97,100,2.10,1009,CO,2024-11-28 04:16:51


### Step 3: Create a new DataFrame called `hotel_df`.

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
hotel_rows = []

for index, row in df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [11]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,Margaret River,AU,-33.9500,115.0667,"Margaret River Hotel, 125 Bussell Highway, Mar...",Margaret River Hotel,681.0,None,https://margaretriverhotel.com.au/
1,Pisco,PE,-13.7000,-76.2167,"La Portada, Calle Alipio Ponce Vasques, Pisco ...",La Portada,1504.0,None,None
2,Zemio,CF,5.0314,25.1361,None,None,NaN,None,None
3,Puerto Ayora,EC,-0.7393,-90.3518,"Hostal Mirada Del Solitario George Hostel, Pet...",Hostal La Mirada De Solitario George,3776.0,None,None
4,Dunedin,NZ,-45.8742,170.5036,"Scenic Hotel Dunedin City, Princes Street, Dun...",Scenic Hotel Dunedin City,240.0,None,https://www.scenichotelgroup.co.nz/
...,...,...,...,...,...,...,...,...,...
69,Botro,CI,7.8525,-5.3106,None,None,NaN,None,None
70,Petatlán,MX,17.5167,-101.2667,None,None,NaN,None,None
71,Guiping,CN,23.3925,110.0814,"西山假日大酒店, 西山路, Guiping City, Guangxi, China",西山假日大酒店,1947.0,None,None
72,Inírida,CO,3.8653,-67.9239,"Residencias El Turista del Guainia, Carrera 2,...",Residencias El Turista del Guainia,790.0,None,http://viktorguarin.blogspot.com.co/


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)